In [10]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [12]:
PROPHET_PATH = '../data/prophet/'
DATA_PATH = '../data/'
RESULTS_PATH = 'results/'

In [13]:
train = pd.read_csv(DATA_PATH+'train_2.csv')

In [14]:
median_df = train.loc[:,['Page']].reset_index()
median_df.columns = ['page_index', 'Page']; median_df.head()

,page_index,Page
0,0,2NE1_zh.wikipedia.org_all-access_spider
1,1,2PM_zh.wikipedia.org_all-access_spider
2,2,3C_zh.wikipedia.org_all-access_spider
3,3,4minute_zh.wikipedia.org_all-access_spider
4,4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...


In [15]:
def prevYear_shift(period):
    year = 365
    assert period[0]
    new1 = period[0] - year
    if not period[1]:
        new2 = -year
    else:
        new2 = period[1] - year
    return (new1,new2)

In [16]:
period = (-60,None)
train.iloc[:,period[0]:period[1]].head()

,2017-07-03,2017-07-04,2017-07-05,2017-07-06,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,26.0,15.0,14.0,15.0,19.0,29.0,22.0,25.0,20.0,19.0,...,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0
1,16.0,23.0,19.0,16.0,816.0,20.0,76.0,24.0,15.0,12.0,...,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0
2,7.0,1.0,1.0,1.0,10.0,1.0,2.0,6.0,5.0,1.0,...,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0
3,16.0,10.0,17.0,15.0,22.0,6.0,6.0,18.0,12.0,16.0,...,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0
4,6.0,16.0,18.0,12.0,14.0,11.0,16.0,15.0,9.0,12.0,...,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0


In [17]:
period = (-60,None)
period = prevYear_shift(period)
train.iloc[:,period[0]:period[1]].head()

,2016-07-03,2016-07-04,2016-07-05,2016-07-06,2016-07-07,2016-07-08,2016-07-09,2016-07-10,2016-07-11,2016-07-12,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15.0,16.0,18.0,21.0,15.0,30.0,115.0,56.0,45.0,17.0,...,20.0,33.0,20.0,14.0,40.0,15.0,18.0,26.0,8.0,25.0
1,12.0,28.0,28.0,17.0,27.0,48.0,184.0,64.0,24.0,92.0,...,29.0,10.0,14.0,23.0,15.0,12.0,25.0,22.0,144.0,31.0
2,2.0,4.0,3.0,4.0,2.0,0.0,1.0,3.0,12.0,4.0,...,4.0,4.0,2.0,5.0,3.0,2.0,3.0,5.0,4.0,2.0
3,11.0,36.0,19.0,35.0,22.0,14.0,17.0,15.0,12.0,34.0,...,5.0,7.0,18.0,9.0,13.0,27.0,15.0,19.0,7.0,9.0
4,3.0,4.0,2.0,3.0,4.0,1.0,1.0,9.0,0.0,1.0,...,2.0,4.0,3.0,3.0,1.0,3.0,5.0,2.0,3.0,2.0


# Median Preds

In [18]:
median_df['median_pred_60'] = np.nan_to_num(np.round(
                        np.nanmedian(
                        train.drop('Page', axis=1).iloc[:,:-60].values[:, -49:], axis=1)).reshape(-1,1)).astype(int)

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [19]:
def preds_period(train, period):
    return np.nan_to_num(np.round(
                        np.nanmedian(
                        train.drop('Page', axis=1).iloc[:,:period[1]].values[:, -49:], axis=1)).reshape(-1,1)).astype(int)

In [20]:
(preds_period(train, (999,-60)) == np.nan_to_num(np.round(
                        np.nanmedian(
                        train.drop('Page', axis=1).iloc[:,:-60].values[:, -49:], axis=1)).reshape(-1,1)).astype(int)).sum()

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


145063

In [21]:
median_df['median_pred_60'] = preds_period(train, (999,-60))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [22]:
median_df['median_pred_120'] = preds_period(train, (999,-120))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [23]:
median_df['prevYear_median_pred_60'] = preds_period(train, prevYear_shift((999,-60)))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [24]:
median_df['prevYear_median_pred_120'] = preds_period(train, prevYear_shift((999,-120)))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [25]:
median_df.head()

,page_index,Page,median_pred_60,median_pred_120,prevYear_median_pred_60,prevYear_median_pred_120
0,0,2NE1_zh.wikipedia.org_all-access_spider,24,23,16,17
1,1,2PM_zh.wikipedia.org_all-access_spider,24,22,14,17
2,2,3C_zh.wikipedia.org_all-access_spider,3,3,4,6
3,3,4minute_zh.wikipedia.org_all-access_spider,15,19,14,14
4,4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,14,14,3,5


# Calculate SMAPES

In [26]:
train2 = train.iloc[:,1:]; train2.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,...,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0
1,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,...,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0
2,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,...,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0
3,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,...,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0


In [27]:
def median_smape_period(train2, preds, period):
    denominator = train2.iloc[:,period[0]:period[1]].add(preds, axis=0) / 200
    diff = np.abs(train2.iloc[:,period[0]:period[1]].subtract(preds, axis=0)) / denominator
    diff[denominator == 0] = 0.0
    #return np.nanmean(diff, axis=0)
    return diff.mean(axis=1, skipna=True)

In [28]:
def smape_60_to_0(train2, median_df):
    denominator = train2.iloc[:,-60:].add(median_df.median_pred_60, axis=0) / 200
    diff = np.abs(train2.iloc[:,-60:].subtract(median_df.median_pred_60, axis=0)) / denominator
    diff[denominator == 0] = 0.0
    #return np.nanmean(diff, axis=0)
    return diff.mean(axis=1, skipna=True)

In [45]:
(smape_60_to_0(train2, median_df) == median_smape_period(train2, median_df.median_pred_60, (-60,None))).sum()

144151

In [30]:
median_df['smape_60_to_0'] = median_smape_period(train2, median_df.median_pred_60, (-60,None))

In [31]:
median_df['smape_60_to_0'].mean()

41.043204039840212

In [32]:
median_df['smape_120_to_60'] = median_smape_period(train2, median_df.median_pred_120, (-120,-60))

In [33]:
median_df['smape_120_to_60'].mean()

40.059152930509576

In [34]:
median_df['prevYear_smape_60_to_0'] = median_smape_period(train2, median_df.prevYear_median_pred_60,
                                                          prevYear_shift((-60,None)))

In [35]:
median_df['prevYear_smape_60_to_0'].mean()

50.551424330915339

In [36]:
median_df['prevYear_smape_120_to_60'] = median_smape_period(train2, median_df.prevYear_median_pred_120,
                                                            prevYear_shift((-120,-60)))

In [37]:
median_df['prevYear_smape_120_to_60'].mean()

47.544453115066389

# NaNs counts

In [38]:
period = (None,-120)
median_df['nans_start_to_120'] = train2.iloc[:,period[0]:period[1]].shape[1] - train2.iloc[:,period[0]:period[1]].count(axis=1)

In [39]:
period = (-120,-60)
median_df['nans_120_to_60'] = train2.iloc[:,period[0]:period[1]].shape[1] - train2.iloc[:,period[0]:period[1]].count(axis=1)

In [40]:
period = (-60,None)
median_df['nans_60_to_0'] = train2.iloc[:,period[0]:period[1]].shape[1] - train2.iloc[:,period[0]:period[1]].count(axis=1)

In [41]:
period = (-60,None)
period = prevYear_shift(period)
median_df['prevYear_nans_60_to_0'] = train2.iloc[:,period[0]:period[1]].shape[1] - train2.iloc[:,period[0]:period[1]].count(axis=1)

In [42]:
period = (-120,-60)
period = prevYear_shift(period)
median_df['prevYear_nans_120_to_60'] = train2.iloc[:,period[0]:period[1]].shape[1] - train2.iloc[:,period[0]:period[1]].count(axis=1)

In [43]:
median_df.head()

,page_index,Page,median_pred_60,median_pred_120,prevYear_median_pred_60,prevYear_median_pred_120,smape_60_to_0,smape_120_to_60,prevYear_smape_60_to_0,prevYear_smape_120_to_60,nans_start_to_120,nans_120_to_60,nans_60_to_0,prevYear_nans_60_to_0,prevYear_nans_120_to_60
0,0,2NE1_zh.wikipedia.org_all-access_spider,24,23,16,17,32.278432,25.959053,40.870388,32.116348,0,0,0,0,0
1,1,2PM_zh.wikipedia.org_all-access_spider,24,22,14,17,42.847863,34.415304,60.431899,32.054860,0,0,0,0,0
2,2,3C_zh.wikipedia.org_all-access_spider,3,3,4,6,64.305367,65.870407,47.163640,56.748585,0,0,0,0,0
3,3,4minute_zh.wikipedia.org_all-access_spider,15,19,14,14,32.250349,33.896551,35.401843,49.278494,0,0,0,0,0
4,4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,14,14,3,5,48.547274,38.718059,54.426324,57.220372,291,0,0,0,0


In [46]:
median_df.to_feather('../data/median_for_120_60.f')